In [1]:
from os.path import dirname
import os, sys, inspect

currentdir = os.getcwd()
parentdir = dirname(currentdir)

sys.path.insert(0,parentdir)

# Libs

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import pickle

# Import Tables

In [3]:
OA_path = '../data/raw/Data OA/'

In [4]:
dump_path = '../data/interim/'

##### Form Admission

In [5]:
form_raw = pd.read_excel(OA_path+'Form Admission.xls',
                            usecols=[2,3,11])

In [6]:
#take rows contain form_number only 
form_master = form_raw[form_raw['form_number'].notnull()]

In [7]:
#format form
form_master['form_number'] = form_master['form_number'].astype(str)
form_master['form_number'] = form_master['form_number'].apply(lambda x: x[:-2])

##### Student

In [8]:
student_raw = pd.read_excel(OA_path+'Student.xls',
                              usecols=[0,7,15,18])

In [9]:
#rename column
student_master = student_raw.rename(columns={
    'id': 'student_id',
    'high_school_id': 'school_id'
})

##### Major

In [10]:
major_raw = pd.read_excel(OA_path+'Major.xls',
                         usecols=[0,7])

In [11]:
#rename column
major_master = major_raw.rename(columns={
    'id': 'major_id',
    'name': 'major_name'
})

##### Gender

In [12]:
gender_raw = pd.read_excel(OA_path+'Gender.xls',
                          usecols=[1,3,4])

In [13]:
gender_master = gender_raw.rename(columns={
    'name': 'gender'
})

In [14]:
gender_master = gender_master[gender_master['language_code']=='en']
gender_master.drop('language_code', axis=1, inplace=True)

##### School

In [15]:
school_raw = pd.read_excel(OA_path+'School Master.xls',
                          usecols=[0,4,10,13])

In [16]:
school_master = school_raw.rename(columns={
    'id': 'school_id',
    'STATE': 'school_state',
    'PROPRIETORSHIP': 'school_prop',
    'DESCR': 'school_name'
})

##### Curriculum

In [17]:
curriculum_raw = pd.read_excel(OA_path+'Curriculum.xls',
                              usecols=[0,3])

In [18]:
curriculum_master = curriculum_raw.rename(columns={
    'id': 'curriculum_id',
    'name': 'curriculum_name'
})

# Merge Tables

In [19]:
#merge Form with Student
OA_master = form_master.merge(student_master, how='left', on='student_id')

In [20]:
#merge with Major
OA_master = OA_master.merge(major_master, how='left', on='major_id')
OA_master.drop('major_id', axis=1, inplace=True)

In [21]:
#merge with Gender
OA_master = OA_master.merge(gender_master, how='left', on='gender_id')
OA_master.drop('gender_id', axis=1, inplace=True)

In [22]:
#merge with School
OA_master = OA_master.merge(school_master, how='left', on='school_id')
OA_master.drop('school_id', axis=1, inplace=True)

In [23]:
#merge with Curriculum
OA_master = OA_master.merge(curriculum_master, how='left', on='curriculum_id')
OA_master.drop('curriculum_id', axis=1, inplace=True)

In [24]:
OA_master.head()

,student_id,form_number,major_name,gender,school_state,school_prop,school_name,curriculum_name
0,4,1811200001,Manajemen,Female,SUMUT,SMA,SMA Negeri 1 Silima Pungga,Other
1,7,1811200002,Arsitektur,Male,SUMBAR,SMA,SMA Don Bosco Padang,International Baccaularate New
2,9,1811200003,Desain Komunikasi Visual,Female,DIY,SMA,SMA Negeri 5 Yogyakarta,International Baccaularate New
3,7,1811100004,Desain Produk,Male,SUMBAR,SMA,SMA Don Bosco Padang,International Baccaularate New
4,15,1811100122,Seni Musik,Female,JATENG,SMA,SMA Negeri 1 Klaten,Science


# Dump to Pickle

In [25]:
pickle.dump(OA_master, open(dump_path+'OA_master.pkl', 'wb'))